In [180]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [233]:
import pandas as pd
raw = pd.read_csv("adult_data_train.csv")
raw_res= pd.read_csv("adult_data_reserved.csv")
raw.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,20,?,122244,HS-grad,9,Never-married,?,Not-in-family,White,Female,0,0,28,United-States,0
3,40,Local-gov,24763,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
4,24,Private,113936,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0


In [182]:
# for c in raw.columns:
#     raw[c] = raw[c].apply(lambda x: raw[c].mode()[0] if x == '?' else x)

In [183]:
raw.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,20,Private,122244,HS-grad,9,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,28,United-States,0
3,40,Local-gov,24763,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
4,24,Private,113936,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0


In [184]:
for c in raw.columns:
    print(c, raw[c].nunique())

age 71
workclass 8
fnlwgt 18437
education 16
education-num 16
marital-status 7
occupation 14
relationship 6
race 5
sex 2
capital-gain 115
capital-loss 88
hours-per-week 94
native-country 41
label 2


In [185]:
pred=raw.drop(['education','marital-status'],axis=1)
pred.isna().sum()

age               0
workclass         0
fnlwgt            0
education-num     0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
label             0
dtype: int64

In [186]:
pred.apply(lambda row: 1 if pd.isna(row['workclass']) or pd.isna(row['occupation']) or pd.isna(row['native-country'])  else 0, axis=1).sum()

0

In [187]:
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education-num   26048 non-null  int64 
 4   occupation      26048 non-null  object
 5   relationship    26048 non-null  object
 6   race            26048 non-null  object
 7   sex             26048 non-null  object
 8   capital-gain    26048 non-null  int64 
 9   capital-loss    26048 non-null  int64 
 10  hours-per-week  26048 non-null  int64 
 11  native-country  26048 non-null  object
 12  label           26048 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 2.6+ MB


In [188]:
pred['label'].value_counts()

label
0    19775
1     6273
Name: count, dtype: int64

In [189]:
pred_num = pred[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week','label']]
pred_num.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label
0,32,37210,13,0,0,45,1
1,43,101950,14,0,0,45,0
2,20,122244,9,0,0,28,0
3,40,24763,10,6849,0,40,0
4,24,113936,13,0,0,40,0


In [190]:
from sklearn.model_selection import train_test_split
train_num_x,test_num_x,train_num_y,test_num_y = train_test_split(pred_num.drop('label',axis=1), pred_num['label'],test_size=0.2, random_state=41,stratify=pred_num['label'])
train_num_x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
4236,33,267859,9,0,0,40
14636,28,230704,9,0,0,40
11353,68,140282,4,0,0,8
11715,26,262617,9,0,0,48
10105,53,260106,15,0,0,50


In [191]:
train_num_x['fnlwgt'].mean()

190199.1213648143

In [192]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()

In [193]:
neigh.fit(train_num_x,train_num_y)

KNeighborsClassifier()

In [194]:
from sklearn.metrics import f1_score
y_pred=neigh.predict(test_num_x)
f1_score(test_num_y,y_pred)

0.3786557208825039

In [195]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
data=scaler.fit_transform(train_num_x)
data_test=scaler.transform(test_num_x)

In [196]:
dataset=pd.DataFrame(data,columns=['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week'])
dataset_test=pd.DataFrame(data_test,columns=['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week'])


In [197]:
dataset['fnlwgt'].mean()

0.12083109531574843

In [198]:
neigh1 = KNeighborsClassifier()
neigh1.fit(dataset,train_num_y)

KNeighborsClassifier()

In [199]:
y_pred=neigh1.predict(dataset_test)
f1_score(test_num_y,y_pred)

0.504655493482309

In [200]:
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education-num   26048 non-null  int64 
 4   occupation      26048 non-null  object
 5   relationship    26048 non-null  object
 6   race            26048 non-null  object
 7   sex             26048 non-null  object
 8   capital-gain    26048 non-null  int64 
 9   capital-loss    26048 non-null  int64 
 10  hours-per-week  26048 non-null  int64 
 11  native-country  26048 non-null  object
 12  label           26048 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 2.6+ MB


In [234]:
# sns.barplot(x=pd.unique(pred['race']).tolist(),y=pred['race'].value_counts())
print(pd.unique(pred['occupation']).tolist())
print(pred['occupation'].value_counts())

['Exec-managerial', 'Prof-specialty', 'Transport-moving', 'Tech-support', 'Craft-repair', 'Sales', 'Other-service', 'Handlers-cleaners', 'Machine-op-inspct', 'Farming-fishing', 'Adm-clerical', 'Protective-serv', 'Priv-house-serv', 'Armed-Forces']
occupation
Prof-specialty       4801
Craft-repair         3299
Exec-managerial      3258
Adm-clerical         2951
Sales                2932
Other-service        2611
Machine-op-inspct    1610
Transport-moving     1275
Handlers-cleaners    1118
Farming-fishing       791
Tech-support          739
Protective-serv       528
Priv-house-serv       129
Armed-Forces            6
Name: count, dtype: int64


In [236]:
pred_new = pred.dropna()
# pred_new.shape, pred.shape, pred_new.shape[0] - pred.shape[0]
pred_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education-num   26048 non-null  int64 
 4   occupation      26048 non-null  object
 5   relationship    26048 non-null  object
 6   race            26048 non-null  object
 7   sex             26048 non-null  object
 8   capital-gain    26048 non-null  int64 
 9   capital-loss    26048 non-null  int64 
 10  hours-per-week  26048 non-null  int64 
 11  native-country  26048 non-null  object
 12  label           26048 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 2.6+ MB


In [ ]:
pred_new.head()

,age,workclass,fnlwgt,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,13,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,14,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,20,Private,122244,9,Prof-specialty,Not-in-family,White,Female,0,0,28,United-States,0
3,40,Local-gov,24763,10,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
4,24,Private,113936,13,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0


In [228]:
[c for c in pred_new.columns if pred_new[c].nunique()]

['age',
 'workclass',
 'fnlwgt',
 'education-num',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [229]:
pred_new_dum=pd.get_dummies(pred_new, columns=['workclass',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country'], drop_first=True).astype(int)

In [ ]:
train_num_dum_x,test_num_dum_x,train_num_dum_y,test_num_dum_y = train_test_split(pred_new_dum.drop('label',axis=1), pred_new_dum['label'],test_size=0.2, random_state=33,stratify=pred_new_dum['label'])
train_num_dum_x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
2479,54,182388,9,0,0,35,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10094,38,177305,9,0,0,45,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1768,32,188557,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
17112,38,365465,11,0,0,15,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
11016,30,54608,10,0,0,48,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
[c for c in pred_new_dum.columns if pred_new_dum[c].nunique()]

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'label',
 'workclass_Local-gov',
 'workclass_Never-worked',
 'workclass_Private',
 'workclass_Self-emp-inc',
 'workclass_Self-emp-not-inc',
 'workclass_State-gov',
 'workclass_Without-pay',
 'occupation_Armed-Forces',
 'occupation_Craft-repair',
 'occupation_Exec-managerial',
 'occupation_Farming-fishing',
 'occupation_Handlers-cleaners',
 'occupation_Machine-op-inspct',
 'occupation_Other-service',
 'occupation_Priv-house-serv',
 'occupation_Prof-specialty',
 'occupation_Protective-serv',
 'occupation_Sales',
 'occupation_Tech-support',
 'occupation_Transport-moving',
 'relationship_Not-in-family',
 'relationship_Other-relative',
 'relationship_Own-child',
 'relationship_Unmarried',
 'relationship_Wife',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Male',
 'native-country_Canada',
 'native-country_China',
 'native-country_Columbia',
 'native-country_Cuba',
 'nat

In [ ]:
scaler1=MinMaxScaler()
data_dum=scaler1.fit_transform(train_num_dum_x)
data_dum_test=scaler1.transform(test_num_dum_x)

In [ ]:
dataset_dum=pd.DataFrame(data_dum,columns=['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_Local-gov',
 'workclass_Never-worked',
 'workclass_Private',
 'workclass_Self-emp-inc',
 'workclass_Self-emp-not-inc',
 'workclass_State-gov',
 'workclass_Without-pay',
 'occupation_Armed-Forces',
 'occupation_Craft-repair',
 'occupation_Exec-managerial',
 'occupation_Farming-fishing',
 'occupation_Handlers-cleaners',
 'occupation_Machine-op-inspct',
 'occupation_Other-service',
 'occupation_Priv-house-serv',
 'occupation_Prof-specialty',
 'occupation_Protective-serv',
 'occupation_Sales',
 'occupation_Tech-support',
 'occupation_Transport-moving',
 'relationship_Not-in-family',
 'relationship_Other-relative',
 'relationship_Own-child',
 'relationship_Unmarried',
 'relationship_Wife',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Male',
 'native-country_Canada',
 'native-country_China',
 'native-country_Columbia',
 'native-country_Cuba',
 'native-country_Dominican-Republic',
 'native-country_Ecuador',
 'native-country_El-Salvador',
 'native-country_England',
 'native-country_France',
 'native-country_Germany',
 'native-country_Greece',
 'native-country_Guatemala',
 'native-country_Haiti',
 'native-country_Holand-Netherlands',
 'native-country_Honduras',
 'native-country_Hong',
 'native-country_Hungary',
 'native-country_India',
 'native-country_Iran',
 'native-country_Ireland',
 'native-country_Italy',
 'native-country_Jamaica',
 'native-country_Japan',
 'native-country_Laos',
 'native-country_Mexico',
 'native-country_Nicaragua',
 'native-country_Outlying-US(Guam-USVI-etc)',
 'native-country_Peru',
 'native-country_Philippines',
 'native-country_Poland',
 'native-country_Portugal',
 'native-country_Puerto-Rico',
 'native-country_Scotland',
 'native-country_South',
 'native-country_Taiwan',
 'native-country_Thailand',
 'native-country_Trinadad&Tobago',
 'native-country_United-States',
 'native-country_Vietnam',
 'native-country_Yugoslavia'])
dataset_dum_test=pd.DataFrame(data_dum_test,columns=['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_Local-gov',
 'workclass_Never-worked',
 'workclass_Private',
 'workclass_Self-emp-inc',
 'workclass_Self-emp-not-inc',
 'workclass_State-gov',
 'workclass_Without-pay',
 'occupation_Armed-Forces',
 'occupation_Craft-repair',
 'occupation_Exec-managerial',
 'occupation_Farming-fishing',
 'occupation_Handlers-cleaners',
 'occupation_Machine-op-inspct',
 'occupation_Other-service',
 'occupation_Priv-house-serv',
 'occupation_Prof-specialty',
 'occupation_Protective-serv',
 'occupation_Sales',
 'occupation_Tech-support',
 'occupation_Transport-moving',
 'relationship_Not-in-family',
 'relationship_Other-relative',
 'relationship_Own-child',
 'relationship_Unmarried',
 'relationship_Wife',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Male',
 'native-country_Canada',
 'native-country_China',
 'native-country_Columbia',
 'native-country_Cuba',
 'native-country_Dominican-Republic',
 'native-country_Ecuador',
 'native-country_El-Salvador',
 'native-country_England',
 'native-country_France',
 'native-country_Germany',
 'native-country_Greece',
 'native-country_Guatemala',
 'native-country_Haiti',
 'native-country_Holand-Netherlands',
 'native-country_Honduras',
 'native-country_Hong',
 'native-country_Hungary',
 'native-country_India',
 'native-country_Iran',
 'native-country_Ireland',
 'native-country_Italy',
 'native-country_Jamaica',
 'native-country_Japan',
 'native-country_Laos',
 'native-country_Mexico',
 'native-country_Nicaragua',
 'native-country_Outlying-US(Guam-USVI-etc)',
 'native-country_Peru',
 'native-country_Philippines',
 'native-country_Poland',
 'native-country_Portugal',
 'native-country_Puerto-Rico',
 'native-country_Scotland',
 'native-country_South',
 'native-country_Taiwan',
 'native-country_Thailand',
 'native-country_Trinadad&Tobago',
 'native-country_United-States',
 'native-country_Vietnam',
 'native-country_Yugoslavia'])

In [ ]:
neigh2 = KNeighborsClassifier()
neigh2.fit(dataset_dum,train_num_dum_y)

KNeighborsClassifier()

In [ ]:
y_pred=neigh2.predict(dataset_dum_test)
f1_score(test_num_dum_y,y_pred)

0.623409669211196